##### Задача 1.
Посчитайте, сколько компаний закрылось.

```sql
SELECT COUNT (status)
FROM company
WHERE status = 'closed'
```

##### Задача 2.
Отобразите количество привлечённых средств для новостных компаний США. Используйте данные из таблицы company. Отсортируйте таблицу по убыванию значений в поле `funding_total`.

```sql
SELECT funding_total
FROM company
WHERE category_code = 'news' AND country_code = 'USA'
ORDER BY funding_total DESC
```

##### Задача 3.
Найдите общую сумму сделок по покупке одних компаний другими в долларах. Отберите сделки, которые осуществлялись только за наличные с 2011 по 2013 год включительно.

```sql
SELECT SUM(price_amount)
FROM acquisition
WHERE term_code = 'cash' 
AND EXTRACT (YEAR FROM CAST(acquired_at AS date)) BETWEEN 2011 AND 2013
```

##### Задача 4.
Отобразите имя, фамилию и названия аккаунтов людей в поле network_username, у которых названия аккаунтов начинаются на `'Silver'`.

```sql
SELECT first_name, 
       last_name, 
       twitter_username
FROM people
WHERE twitter_username LIKE 'Silver%'
```

##### Задача 5.
Выведите на экран всю информацию о людях, у которых названия аккаунтов в поле network_username содержат подстроку `'money'`, а фамилия начинается на `'K'`.

```sql
SELECT *
FROM people
WHERE twitter_username LIKE '%money%'
AND last_name LIKE 'K%'
```

##### Задача 6.
Для каждой страны отобразите общую сумму привлечённых инвестиций, которые получили компании, зарегистрированные в этой стране. Страну, в которой зарегистрирована компания, можно определить по коду страны. Отсортируйте данные по убыванию суммы.

```sql
SELECT country_code, 
       SUM(funding_total)
FROM company
GROUP BY country_code
ORDER BY SUM(funding_total) DESC
```

##### Задача 7.
Составьте таблицу, в которую войдёт дата проведения раунда, а также минимальное и максимальное значения суммы инвестиций, привлечённых в эту дату.
Оставьте в итоговой таблице только те записи, в которых минимальное значение суммы инвестиций не равно нулю и не равно максимальному значению.

```sql
SELECT funded_at, 
       MIN(raised_amount), 
       MAX(raised_amount)
FROM funding_round
GROUP BY funded_at
HAVING MIN(raised_amount) != 0 AND MIN(raised_amount) != MAX(raised_amount)
```

##### Задача 8.
Создайте поле с категориями:
- Для фондов, которые инвестируют в 100 и более компаний, назначьте категорию `high_activity`.
- Для фондов, которые инвестируют в 20 и более компаний до 100, назначьте категорию `middle_activity`.
- Если количество инвестируемых компаний фонда не достигает 20, назначьте категорию `low_activity`.

Отобразите все поля таблицы fund и новое поле с категориями.

```sql
SELECT *,
      CASE
           WHEN invested_companies < 20 THEN 'low_activity'
           WHEN invested_companies < 100 THEN 'middle_activity'
           WHEN invested_companies >= 100 THEN 'high_activity'
       END
FROM fund
```

##### Задача 9.
Для каждой из категорий, назначенных в предыдущем задании, посчитайте округлённое до ближайшего целого числа среднее количество инвестиционных раундов, в которых фонд принимал участие. Выведите на экран категории и среднее число инвестиционных раундов. Отсортируйте таблицу по возрастанию среднего.

```sql
SELECT 
CASE
   WHEN invested_companies>=100 THEN 'high_activity'
   WHEN invested_companies>=20 THEN 'middle_activity'
   ELSE 'low_activity'
END AS activity,
ROUND(AVG(investment_rounds))
FROM fund
GROUP BY activity
ORDER BY ROUND(AVG(investment_rounds));
```

##### Задача 10.
Проанализируйте, в каких странах находятся фонды, которые чаще всего инвестируют в стартапы. 

Для каждой страны посчитайте минимальное, максимальное и среднее число компаний, в которые инвестировали фонды этой страны, основанные с 2010 по 2012 год включительно. Исключите страны с фондами, у которых минимальное число компаний, получивших инвестиции, равно нулю.
 
Выгрузите десять самых активных стран-инвесторов: отсортируйте таблицу по среднему количеству компаний от большего к меньшему. Затем добавьте сортировку по коду страны в лексикографическом порядке.

```sql
SELECT country_code, 
       MIN(invested_companies),
       MAX(invested_companies),
       AVG(invested_companies)
FROM fund
WHERE EXTRACT(YEAR FROM CAST(founded_at AS date)) BETWEEN 2010 AND 2012
GROUP BY country_code
HAVING MIN(invested_companies) > 0
ORDER BY AVG(invested_companies) DESC, country_code
LIMIT 10
```

##### Задача 11.
Отобразите имя и фамилию всех сотрудников стартапов. Добавьте поле с названием учебного заведения, которое окончил сотрудник, если эта информация известна.

```sql
SELECT first_name, last_name, instituition
FROM people
LEFT JOIN education ON people.id = education.person_id
```

##### Задача 12.
Для каждой компании найдите количество учебных заведений, которые окончили её сотрудники. Выведите название компании и число уникальных названий учебных заведений. Составьте топ-5 компаний по количеству университетов.

```sql
SELECT company.name,       
COUNT(DISTINCT(instituition))
FROM company
LEFT JOIN people ON company.id = people.company_id
LEFT JOIN education ON people.id = education.person_id
GROUP BY company.id
ORDER BY COUNT(DISTINCT(instituition)) DESC
LIMIT 5
```

##### Задача 13.
Составьте список с уникальными названиями закрытых компаний, для которых первый раунд финансирования оказался последним.

```sql
SELECT DISTINCT(company.name)
FROM funding_round
LEFT JOIN company ON company.id = funding_round.company_id
WHERE is_first_round = 1 AND is_last_round = 1
AND company.status LIKE 'closed'
```

##### Задача 14.
Составьте список уникальных номеров сотрудников, которые работают в компаниях, отобранных в предыдущем задании.

```sql
WITH org AS (
SELECT DISTINCT(company.id)
FROM funding_round
LEFT JOIN company ON company.id = funding_round.company_id
WHERE is_first_round = 1 AND is_last_round = 1
AND company.status LIKE 'closed'
)

SELECT DISTINCT(people.id)
FROM people
WHERE people.company_id IN (SELECT * FROM org)
```

##### Задача 15.
Составьте таблицу, куда войдут уникальные пары с номерами сотрудников из предыдущей задачи и учебным заведением, которое окончил сотрудник.

```sql
WITH org AS (
SELECT DISTINCT(company.id)
FROM funding_round
LEFT JOIN company ON company.id = funding_round.company_id
WHERE is_first_round = 1 AND is_last_round = 1
AND company.status LIKE 'closed'
),

p_id AS(
SELECT DISTINCT(people.id)
FROM people
WHERE people.company_id IN (SELECT * FROM org)
)    

SELECT person_id, instituition
FROM education
WHERE person_id IN (SELECT * FROM p_id)
UNION
SELECT person_id, instituition
FROM education
WHERE person_id IN (SELECT * FROM p_id)
```

##### Задача 16.
Посчитайте количество учебных заведений для каждого сотрудника из предыдущего задания. При подсчёте учитывайте, что некоторые сотрудники могли окончить одно и то же заведение дважды.

```sql
WITH org AS (
SELECT DISTINCT(company.id)
FROM funding_round
LEFT JOIN company ON company.id = funding_round.company_id
WHERE is_first_round = 1 AND is_last_round = 1
AND company.status LIKE 'closed'
),

p_id AS (
SELECT DISTINCT(people.id)
FROM people
WHERE people.company_id IN (SELECT * FROM org)
)

SELECT person_id, COUNT(instituition)
FROM education
WHERE person_id IN (SELECT * FROM p_id)
GROUP BY person_id
```

##### Задача 17.
Дополните предыдущий запрос и выведите среднее число учебных заведений (всех, не только уникальных), которые окончили сотрудники разных компаний. Нужно вывести только одну запись, группировка здесь не понадобится.

```sql
WITH org AS (
SELECT DISTINCT(company.id)
FROM funding_round
LEFT JOIN company ON company.id = funding_round.company_id
WHERE is_first_round = 1 AND is_last_round = 1
AND company.status LIKE 'closed'
),

p_id AS (
SELECT DISTINCT(people.id)
FROM people
WHERE people.company_id IN (SELECT * FROM org)
)

SELECT AVG(count)
FROM(
SELECT person_id, COUNT(instituition)
FROM education
WHERE person_id IN (SELECT * FROM p_id)
GROUP BY person_id) AS tab
```

##### Задача 18.
Напишите похожий запрос: выведите среднее число учебных заведений (всех, не только уникальных), которые окончили сотрудники Facebook*.

*(сервис, запрещённый на территории РФ)

```sql
WITH org AS (
SELECT DISTINCT(company.id)
FROM funding_round
LEFT JOIN company ON company.id = funding_round.company_id
WHERE company.name LIKE 'Facebook'
),
    
p_id AS
(
SELECT DISTINCT(people.id)
FROM people
WHERE people.company_id IN (SELECT * FROM org)
)

SELECT AVG(count)
FROM
(
SELECT person_id, COUNT(instituition)
FROM education
WHERE person_id IN (SELECT * FROM p_id)
GROUP BY person_id
) AS tab
```

##### Задача 19.

Составьте таблицу из полей:

- `name_of_fund` — название фонда;
- `name_of_company` — название компании;
- `amount` — сумма инвестиций, которую привлекла компания в раунде.

В таблицу войдут данные о компаниях, в истории которых было больше шести важных этапов, а раунды финансирования проходили с 2012 по 2013 год включительно.

```sql
SELECT f.name AS name_of_fund,
       com.name AS name_of_company,
       fr.raised_amount AS amount
FROM investment AS i
INNER JOIN company AS com ON i.company_id=com.id
INNER JOIN fund AS f ON i.fund_id=f.id
INNER JOIN funding_round AS fr ON i.funding_round_id=fr.id
WHERE com.milestones > 6
   AND EXTRACT(YEAR FROM CAST (fr.funded_at AS TIMESTAMP)) BETWEEN 2012 AND 2013
```

##### Задача 20.

Выгрузите таблицу, в которой будут такие поля:

- название компании-покупателя;
- сумма сделки;
- название компании, которую купили;
- сумма инвестиций, вложенных в купленную компанию;
- доля, которая отображает, во сколько раз сумма покупки превысила сумму вложенных в компанию инвестиций, округлённая до ближайшего целого числа.

Не учитывайте те сделки, в которых сумма покупки равна нулю. Если сумма инвестиций в компанию равна нулю, исключите такую компанию из таблицы. 
Отсортируйте таблицу по сумме сделки от большей к меньшей, а затем по названию купленной компании в лексикографическом порядке. Ограничьте таблицу первыми десятью записями.

```sql
SELECT c_acquiring.name, 
       acquisition.price_amount, 
       c_acquired.name,     
       c_acquired.funding_total, 
       ROUND(acquisition.price_amount/c_acquired.funding_total)
FROM acquisition
LEFT JOIN company AS c_acquiring ON c_acquiring.id = acquisition.acquiring_company_id
LEFT JOIN company AS c_acquired ON c_acquired.id = acquisition.acquired_company_id
WHERE acquisition.price_amount > 0 AND c_acquired.funding_total > 0
ORDER BY acquisition.price_amount DESC, c_acquired.name
LIMIT 10
```

##### Задача 21.
Выгрузите таблицу, в которую войдут названия компаний из категории `social`, получившие финансирование с 2010 по 2013 год включительно. Проверьте, что сумма инвестиций не равна нулю. Выведите также номер месяца, в котором проходил раунд финансирования.

```sql
SELECT company.name, 
EXTRACT(MONTH FROM CAST(funding_round.funded_at AS date))
FROM company
LEFT JOIN funding_round ON  company.id = funding_round.company_id
WHERE company.category_code LIKE 'social'
AND EXTRACT(YEAR FROM CAST(funding_round.funded_at AS date)) BETWEEN 2010 AND 2013
AND funding_round.raised_amount > 0
```

##### Задача 22.
Отберите данные по месяцам с 2010 по 2013 год, когда проходили инвестиционные раунды. Сгруппируйте данные по номеру месяца и получите таблицу, в которой будут поля:

- номер месяца, в котором проходили раунды;
- количество уникальных названий фондов из США, которые инвестировали в этом месяце;
- количество компаний, купленных за этот месяц;
- общая сумма сделок по покупкам в этом месяце.

```sql

```

##### Задача 23.
Посчитайте, сколько компаний закрылось.

```sql

```